# 4. DETECTION ET ANALYSE DES OUTLIERS


### Objectif : 
#### Identifier les valeurs extrêmes qui peuvent fausser les statistiques et les modèles.

## A) IMPORT DES DATAS

In [3]:
import pandas as pd

df = pd.read_csv("../6.Data/kaggle_b2_fraud_train_v3.csv")

In [4]:
df.head()

,customer_id,account_id,age,tenure_months,annual_income_eur,credit_score,num_transactions_30d,avg_amount_30d_eur,max_amount_30d_eur,days_since_last_login,...,internal_signal_5,internal_signal_6,internal_signal_7,internal_signal_8,terms_accepted_flag,partner_risk_indicator,manual_review_result,post_event_status_code,chargeback_resolution_time_days,legacy_partner_score
0,CUST_6O9Q8D4I36,ACC_TXXXTNEUVKFY,34,108,38635.01,544.0,20,60.92,80.16,4.9,...,0.39006,0.10963,0.55097,-0.56104,1,NaN,approve,0,7.9,NaN
1,CUST_FGUGTW230C,ACC_70VD7A4FFWCW,48,2,19912.97,703.0,21,112.11,571.12,0.3,...,0.03265,-0.40256,0.36218,0.86583,1,NaN,approve,0,5.5,NaN
2,CUST_8ZI3LCBZ0W,ACC_AF53381QSC0L,27,0,20326.87,720.0,25,73.61,492.57,4.6,...,-0.15637,0.57818,0.28902,-2.19864,1,NaN,approve,0,7.2,NaN
3,CUST_5MP3AR41CJ,ACC_U7WZGJ486LIV,45,49,38452.47,703.0,17,47.53,204.18,25.3,...,-1.02145,0.63908,-0.89190,-0.81592,1,NaN,approve,0,4.4,NaN
4,CUST_GNPL83JB0J,ACC_XW7DS3ED5J4Y,37,46,NaN,594.0,13,99.95,734.09,12.8,...,-0.65771,0.08020,0.17606,0.86739,1,NaN,approve,0,4.9,NaN


## B) Identifier les colonnes concernées

In [5]:
#On garde que les colonnes numériques
df_numerique = df.select_dtypes(include='number')

#On Garde que les colonnes numériques qui peuvent avoir des outliers , ou qui ne sont pas catégorielles avec des chiffres
colonne_a_enlever = ['is_vpn','is_new_device','postal_code','target_is_fraud','post_event_status_code']

# DataFrame sans ces colonnes
df_sans = df_numerique.drop(columns=colonne_a_enlever)

# DataFrame avec seulement ces colonnes
df_avec = df_numerique[colonne_a_enlever]
df_sans

,age,tenure_months,annual_income_eur,credit_score,num_transactions_30d,avg_amount_30d_eur,max_amount_30d_eur,days_since_last_login,support_tickets_90d,chargebacks_12m,...,internal_signal_3,internal_signal_4,internal_signal_5,internal_signal_6,internal_signal_7,internal_signal_8,terms_accepted_flag,partner_risk_indicator,chargeback_resolution_time_days,legacy_partner_score
0,34,108,38635.01,544.0,20,60.92,80.16,4.9,1,0,...,-0.68676,-1.54627,0.39006,0.10963,0.55097,-0.56104,1,NaN,7.9,NaN
1,48,2,19912.97,703.0,21,112.11,571.12,0.3,0,0,...,-0.17429,-0.00054,0.03265,-0.40256,0.36218,0.86583,1,NaN,5.5,NaN
2,27,0,20326.87,720.0,25,73.61,492.57,4.6,1,0,...,-0.86064,-0.32183,-0.15637,0.57818,0.28902,-2.19864,1,NaN,7.2,NaN
3,45,49,38452.47,703.0,17,47.53,204.18,25.3,1,0,...,-1.11761,1.35968,-1.02145,0.63908,-0.89190,-0.81592,1,NaN,4.4,NaN
4,37,46,NaN,594.0,13,99.95,734.09,12.8,0,1,...,0.11581,1.65795,-0.65771,0.08020,0.17606,0.86739,1,NaN,4.9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,56,0,34775.62,727.0,21,51.72,226.11,3.8,0,0,...,-0.42671,0.35144,-2.54086,-0.60747,0.23252,-0.06215,1,-1.535486,1.0,NaN
159996,41,4,88617.57,770.0,18,NaN,171.07,15.1,1,0,...,2.42123,0.33237,0.34098,-1.78817,0.31788,0.51072,1,NaN,7.4,NaN
159997,30,2,41148.54,738.0,20,29.34,119.81,0.7,0,0,...,0.63827,0.84880,-1.28947,-0.32324,-0.06238,-0.99076,1,NaN,6.6,NaN
159998,56,6,NaN,719.0,25,88.56,553.16,22.6,2,0,...,0.79633,-1.87471,0.47179,-0.22090,-1.34239,-0.30513,1,NaN,12.5,NaN


## C) Detection des Outliers avec l’IQR

In [16]:
Q1 = df_sans.quantile(0.25)
Q3 = df_sans.quantile(0.75)
IQR = Q3 - Q1

borne_inf = Q1 - 2 * IQR
borne_sup = Q3 + 2 * IQR

outliers_iqr = (df_sans < borne_inf) | (df_sans > borne_sup)
iqr_counts = outliers_iqr.sum(axis=0)
iqr_counts

age                                 210
tenure_months                      4054
annual_income_eur                  3695
credit_score                         85
num_transactions_30d                316
avg_amount_30d_eur                 2184
max_amount_30d_eur                 3931
days_since_last_login              4449
support_tickets_90d                1494
chargebacks_12m                    7802
failed_payments_6m                   72
device_trust_z                        0
ip_risk_z                             0
num_devices_30d                    1631
income_log                          101
income_estimate_alt_eur            3778
credit_score_norm                    82
tx_amount_total_30d_eur            3038
max_to_avg_ratio                      0
internal_signal_1                   132
internal_signal_2                   116
internal_signal_3                   131
internal_signal_4                   122
internal_signal_5                   129
internal_signal_6                   123


## D) Detection des Outliers avec le Z-score

In [17]:
from scipy import stats
import numpy as np

z_scores = np.abs(stats.zscore(df_sans, nan_policy='omit')) 
outliers_z = (z_scores > 3)
z_counts = outliers_z.sum(axis=0)
z_counts

C:\Users\yannc\AppData\Local\Temp\ipykernel_14368\3227689761.py:4: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = np.abs(stats.zscore(df_sans, nan_policy='omit'))


array([ 407, 2927, 1475,  236,  127, 2056, 2613, 2927, 1494, 7802,  834,
        427,  427, 1631,  438, 2625,  495, 2442,    0,  432,  432,  466,
        415,  438,  486,  417,  438,    0,   14, 3272,   14])

In [18]:

# --- Créer le DataFrame résumé ---
df_outliers_summary = pd.DataFrame({
    'colonne': df_sans.columns,
    'iqr_outliers': iqr_counts.values,
    'zscore_outliers': z_counts
})
df_outliers_summary['TOTAL_OUTLIERS'] = df_outliers_summary['iqr_outliers'] + df_outliers_summary['zscore_outliers']
# Afficher
df_outliers_summary.sort_values(by='TOTAL_OUTLIERS', ascending=False)


,colonne,iqr_outliers,zscore_outliers,TOTAL_OUTLIERS
9,chargebacks_12m,7802,7802,15604
7,days_since_last_login,4449,2927,7376
29,chargeback_resolution_time_days,3853,3272,7125
1,tenure_months,4054,2927,6981
6,max_amount_30d_eur,3931,2613,6544
15,income_estimate_alt_eur,3778,2625,6403
17,tx_amount_total_30d_eur,3038,2442,5480
2,annual_income_eur,3695,1475,5170
5,avg_amount_30d_eur,2184,2056,4240
13,num_devices_30d,1631,1631,3262


## E) Analyse des Extreme outliers


In [ ]:
from scipy import stats
import numpy as np

z_scores = np.abs(stats.zscore(df_sans, nan_policy='omit')) 
outliers_z = (z_scores > 6)
z_counts = outliers_z.sum(axis=0)
z_counts

Q1 = df_sans.quantile(0.25)
Q3 = df_sans.quantile(0.75)
IQR = Q3 - Q1

borne_inf = Q1 - 4 * IQR
borne_sup = Q3 + 4 * IQR

outliers_iqr = (df_sans < borne_inf) | (df_sans > borne_sup)
iqr_counts = outliers_iqr.sum(axis=0)
iqr_counts


# --- Créer le DataFrame résumé ---
df_outliers_summary = pd.DataFrame({
    'colonne': df_sans.columns,
    'iqr_outliers': iqr_counts.values,
    'zscore_outliers': z_counts
})

df_outliers_summary['TOTAL_OUTLIERS'] = df_outliers_summary['iqr_outliers'] + df_outliers_summary['zscore_outliers']

df_outliers_summary.sort_values(by='TOTAL_OUTLIERS', ascending=False)


C:\Users\yannc\AppData\Local\Temp\ipykernel_14368\1870559332.py:4: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = np.abs(stats.zscore(df_sans, nan_policy='omit'))


,colonne,iqr_outliers,zscore_outliers,TOTAL_OUTLIERS
9,chargebacks_12m,7802,183,7985
29,chargeback_resolution_time_days,1745,534,2279
2,annual_income_eur,665,225,890
15,income_estimate_alt_eur,521,184,705
7,days_since_last_login,501,143,644
6,max_amount_30d_eur,420,134,554
1,tenure_months,490,0,490
17,tx_amount_total_30d_eur,223,91,314
4,num_transactions_30d,127,127,254
5,avg_amount_30d_eur,99,55,154


## F) Analyser et recommendation

| Colonne                           | Rôle / Signification                            | TOTAL_OUTLIERS | Faut-il s’inquiéter ?                                             | Comment gérer                                                                             |
| --------------------------------- | ----------------------------------------------- | -------------- | ----------------------------------------------------------------- | ----------------------------------------------------------------------------------------- |
| `chargebacks_12m`                 | Nombre de rétrofacturations passées sur 12 mois | 15 604         | Oui, mais ce sont des valeurs critiques pour le signal de fraude  | Conserver tous les outliers, éventuellement clip très haut si valeurs aberrantes extrêmes |
| `days_since_last_login`           | Jours depuis dernière connexion                 | 7 376          | Oui, peut indiquer inactivité anormale                            | Clip aux percentiles 1–99 ou standardisation après transformation log                     |
| `chargeback_resolution_time_days` | Temps moyen résolution rétrofacturations        | 7 125          | Oui, valeurs extrêmes peuvent refléter des comptes problématiques | Clip ou log-transform si distribution très skewed                                         |
| `tenure_months`                   | Ancienneté du compte                            | 6 981          | Modérément, mais souvent long / récent impacte peu                | Clip aux bornes plausibles ou standardisation                                             |
| `max_amount_30d_eur`              | Montant maximal transaction sur 30 jours        | 6 544          | Oui, montants extrêmes peuvent influencer modèle                  | Log-transform ou winsorisation                                                            |
| `income_estimate_alt_eur`         | Estimation alternative du revenu                | 6 403          | Oui, valeurs extrêmes possibles                                   | Log-transform ou winsorisation                                                            |
| `tx_amount_total_30d_eur`         | Montant total transactions 30 jours             | 5 480          | Oui, signal important                                             | Log-transform ou clip 1–99 percentile                                                     |
| `annual_income_eur`               | Revenu annuel estimé                            | 5 170          | Oui, distribution skewed                                          | Log-transform ou winsorisation                                                            |
| `avg_amount_30d_eur`              | Montant moyen transaction 30 jours              | 4 240          | Oui, valeurs extrêmes influencent modèle                          | Log-transform ou clip                                                                     |
| `num_devices_30d`                 | Nombre d’appareils utilisés                     | 3 262          | Oui, très élevé peut indiquer fraude                              | Clip aux percentiles 1–99 ou log-transform                                                |
| `support_tickets_90d`             | Tickets support ouverts                         | 2 988          | Oui, trop de tickets peut signaler problème                       | Clip aux percentiles 1–99                                                                 |
| `failed_payments_6m`              | Paiements échoués sur 6 mois                    | 906            | Oui mais faible nombre → signal pertinent                         | Conserver, clip max si extrêmes aberrants                                                 |
| `age`                             | Âge du client                                   | 617            | Modéré                                                            | Clip aux bornes 18–99 ans                                                                 |
| `internal_signal_6`               | Signal interne opérationnel                     | 609            | Oui, valeurs extrêmes peuvent signaler anomalie                   | Standardisation ±3 sigma                                                                  |
| `internal_signal_3`               | Signal interne                                  | 597            | Oui                                                               | Standardisation ±3 sigma                                                                  |
| `credit_score_norm`               | Score crédit normalisé                          | 577            | Oui, influence le scoring                                         | Standardisation ±3 sigma                                                                  |
| `internal_signal_5`               | Signal interne                                  | 567            | Oui                                                               | Standardisation ±3 sigma                                                                  |
| `internal_signal_1`               | Signal interne                                  | 564            | Oui                                                               | Standardisation ±3 sigma                                                                  |
| `internal_signal_8`               | Signal interne                                  | 552            | Oui                                                               | Standardisation ±3 sigma                                                                  |
| `internal_signal_2`               | Signal interne                                  | 548            | Oui                                                               | Standardisation ±3 sigma                                                                  |
| `internal_signal_7`               | Signal interne                                  | 545            | Oui                                                               | Standardisation ±3 sigma                                                                  |
| `income_log`                      | Revenu log-transformé                           | 539            | Non préoccupant                                                   | Conserver tel quel                                                                        |
| `internal_signal_4`               | Signal interne                                  | 537            | Oui                                                               | Standardisation ±3 sigma                                                                  |
| `num_transactions_30d`            | Nb transactions 30 jours                        | 443            | Oui, mais peu préoccupant                                         | Standardisation ±3 sigma ou clip                                                          |
| `ip_risk_z`                       | Score risque IP                                 | 427            | Modéré                                                            | Standardisation ±3 sigma                                                                  |
| `device_trust_z`                  | Score confiance device                          | 427            | Modéré                                                            | Standardisation ±3 sigma                                                                  |
| `credit_score`                    | Score crédit                                    | 321            | Oui mais faible                                                   | Clip ±3 sigma                                                                             |
| `legacy_partner_score`            | Score externe partenaire                        | 20             | Non préoccupant                                                   | Conserver                                                                                 |
| `partner_risk_indicator`          | Indicateur risque partenaire                    | 19             | Non préoccupant                                                   | Conserver                                                                                 |
| `max_to_avg_ratio`                | Ratio max/moyenne transaction                   | 0              | Non                                                               | Conserver                                                                                 |
| `terms_accepted_flag`             | Acceptation conditions                          | 0              | Non                                                               | Conserver                                                                                 |


| Action recommandée                            | Colonnes concernées                                                                                                                                                                                                                                                                                          |
| --------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **Ne pas toucher / conserver telles quelles** | `income_log`, `legacy_partner_score`, `partner_risk_indicator`, `max_to_avg_ratio`, `terms_accepted_flag`                                                                                                                                                                                                    |
| **Standardisation ±3 sigma**                  | `internal_signal_6`, `internal_signal_3`, `credit_score_norm`, `internal_signal_5`, `internal_signal_1`, `internal_signal_8`, `internal_signal_2`, `internal_signal_7`, `internal_signal_4`, `num_transactions_30d`, `ip_risk_z`, `device_trust_z`                                                           |
| **Log-transform / Winsorisation / Clip**      | `chargebacks_12m`, `days_since_last_login`, `chargeback_resolution_time_days`, `tenure_months`, `max_amount_30d_eur`, `income_estimate_alt_eur`, `tx_amount_total_30d_eur`, `annual_income_eur`, `avg_amount_30d_eur`, `num_devices_30d`, `support_tickets_90d`, `failed_payments_6m`, `age`, `credit_score` |
